__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/bludwig/blob/main/jupyter/52_agnews_text_classification/_Train_agnews.ipynb)__

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install --quiet blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Dropbox
environment['lib_path']     = /home/me/Dropbox/31_Projekte/01_Python/libs
Start Time: 22:08:04


# Train agnews Text Classification
https://colab.research.google.com/github/ludwig-ai/ludwig-docs/blob/master/docs/examples/llm/LLM_fine_tuning_for_Text_Classification_with_Ludwig_Python_API.ipynb#scrollTo=FczUptADgreZ

In [2]:
# Settings

mode            = 'T'                     # mode: C=cleanup, T=train, S=show 
train_jobs      = [ 0, 1,  ]      # Welche models sollen trainiert werden?
verbose         = False   

dataset_name    = 'agnews'                # Which dataset shall be loaded?
experiment_name = 'agnews'                # Base of the model names

predicts_size   = 100              # Wie viele Predictions sollen erstellt werden?
predict_model   =   1              # Mit welchem model sollen predictions erstellt werden?
   
gdrive_base     = 'drive/MyDrive/Colab'
bgc('Beige')

In [3]:
# Colab? >> Install the latest version of Ludwig
if environment['in_colab']:
  !pip uninstall -y tensorflow --quiet
  !python -m pip install git+https://github.com/ludwig-ai/ludwig.git --quiet 
  !pip install ptitprince --quiet 
  !pip install bludwig --quiet 

In [4]:
# cleanup?
if mode == 'C': 
    try:
        !rm -rf results
        !rm *.meta.json
        !rm *.hdf5    
        !rm *.temp.pickle        
        !rm .lock_preprocessing
        !rm train_log*.csv
        !rm datasets.csv        
    except:
        pass
 

if mode == 'C':
    raise Stop
else:
    bgc('WhiteSmoke')

In [5]:
# import 
import pandas as pd
import pandasklar as pak
grid = pak.grid
import bludwig

## Data

In [6]:
# load_dataset
data_df, dataset_loader = bludwig.load_dataset(dataset_name)


agnews 1.0
News articles categorized as "World", "Sports", "Business", and "Science".

output_features: [{'name': 'class', 'type': 'category'}]


In [7]:
# examine
print(data_df.shape[0], 'records for training, validation and test')
pak.sample(data_df,10)

127598 records for training, validation and test


,class,title,description,split
0,business,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,1
491,world,Venezuelans Vote Early in Referendum on Chavez...,Reuters - Venezuelans turned out early\and in ...,1
1157,sci_tech,#147;Generic Superheroes #148;,"Scripted, produced, cast, costumed, shot, edit...",1
4379,sci_tech,Enter your e-mail:,System administrators who have been installing...,0
14371,business,"SUSAN TOMPOR: Change the law, feel the bounce","you know, writing a check a day or two before ...",0
35155,sci_tech,worm has turned for teen virus king,"Sven Jaschan, 18, from Germany, faces up to fi...",0
38967,sports,The Rundown,1 Southern California at Stanford &lt;em&...,0
82435,business,Cazenove Loses Independence with JPM Deal,LONDON (Reuters) - UK brokerage Cazenove effe...,0
95511,world,US expected to boost troop levels in Iraq,Some question whether it will be enough to que...,0
127597,business,EBay gets into rentals,EBay plans to buy the apartment and home renta...,2


In [8]:
# analyse_cols
analyse = bludwig.analyse_cols(data_df, dataset_loader)
analyse

,col_name,is_output_feature,feature_type,datatype_short,datatype_identified,mem_usage,nunique,ndups,vmin,vmax,n
0,class,True,category,string,string,634,4,9996,business,world,10000
1,title,False,text,string,string,953,9945,55,#147;Generic Superheroes #148;,worm has turned for teen virus king,10000
2,description,False,text,string,string,2538,9980,20,1 Southern California at Stanford &lt;em&...,"you know, writing a check a day or two before ...",10000


## configs

In [9]:
# Automatically generated / provided configs
configs = bludwig.configs(data_df, dataset_loader)

In [10]:
# embed

config_1 = """

input_features:
        
  - name: description
    type: text
    encoder: 
        type: embed
      

output_features:

  - name: class
    type: category

"""

In [11]:
# parallel_cnn

config_2 = """

input_features:
  - name: description
    type: text
    encoder:
      type: parallel_cnn
      
output_features:
  - name: class
    type: category
    
trainer:
  epochs: 5


"""

In [12]:
# LLM distilbert1
# colab: 45 min
# GPU:    3 std!

# We are using distilbert because it is small enough to run a few epochs in colab. 
# Bigger models like BERT, RoBERTa, T5, can get better results, but may timeout. 
# Look at the end of the notebook for an example of how to train very large models like Bloom using multiple GPUs.

config_3 = """

input_features:
  - name: description
    type: text
    encoder:
      type: distilbert
      trainable: true
      
output_features:
  - name: class
    type: category
    
trainer:
  epochs: 3
  learning_rate: 0.00001
  optimizer:
    type: adamw
  use_mixed_precision: true
  learning_rate_scheduler:
    decay: linear
    warmup_fraction: 0.2
  batch_size: 32
  
"""

In [13]:
# LLM distilbert2
# colab: 30 min
# GPU:   45 min

# Freeze distilbert weights, and use cached embeddings to speed up training.
# Faster training allows us to train for more epochs, 
# though we do see a hit to performance compared to fine-tuning weights.

config_4 = """

input_features:
  - name: description
    type: text
    encoder:
      type: distilbert
      trainable: false
    preprocessing:
      cache_encoder_embeddings: true
      
combiner:
  type: concat
  num_fc_layers: 2
  
output_features:
  - name: class
    type: category
    
trainer:
  epochs: 30
  learning_rate: 0.00001
  optimizer:
    type: adamw
  use_mixed_precision: true
  learning_rate_scheduler:
    decay: linear
    warmup_fraction: 0.2
  batch_size: 32
  
"""

In [14]:
# add configs
configs += [ config_1, config_2, config_3, config_4 ]

for i, c in enumerate(configs):
    print('\n====== config for model {} ======\n'.format( experiment_name + '_' + str(i)))
    print(c)
    print('\n\n')


====== config for model agnews_0 ======

input_features:
- name: title
  type: text
- name: description
  type: text
output_features:
- name: class
  type: category





====== config for model agnews_1 ======



input_features:
        
  - name: description
    type: text
    encoder: 
        type: embed
      

output_features:

  - name: class
    type: category






====== config for model agnews_2 ======



input_features:
  - name: description
    type: text
    encoder:
      type: parallel_cnn
      
output_features:
  - name: class
    type: category
    
trainer:
  epochs: 5







====== config for model agnews_3 ======



input_features:
  - name: description
    type: text
    encoder:
      type: distilbert
      trainable: true
      
output_features:
  - name: class
    type: category
    
trainer:
  epochs: 3
  learning_rate: 0.00001
  optimizer:
    type: adamw
  use_mixed_precision: true
  learning_rate_scheduler:
    decay: linear
    warmup_fraction: 0.2
  batc

## Initialize

In [16]:
# bludwig initialize
if mode == 'T':  
    ludwig_job = bludwig.LudwigJob( configs=configs, experiment_name=experiment_name, verbose=verbose) 


5 configs loaded


In [17]:
# bludwig initialize and load
if mode == 'S':  
    ludwig_job = bludwig.LudwigJob() 
    ludwig_job.load_from_results()
    #print(ludwig_job)

## Train

In [18]:
# train and evaluate
out = None
if mode == 'T':  
    ludwig_job.experiment(data_df)
    out = bludwig.train_log_to_csv( ludwig_job.train_log )
else:
    out = 'No training'
out


Training model agnews_0

Received SIGINT, will finish this training step and then conclude training.
Send another SIGINT to immediately interrupt the process.

Received a second SIGINT, will now quit


SystemExit: 1

/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3450: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



## Evaluate

In [ ]:
# compare_performance
ludwig_job.compare_performance()

In [ ]:
# learning_curves
ludwig_job.learning_curves()

In [ ]:
# confusion_matrix
#ludwig_job.load_model(0)
ludwig_job.confusion_matrix()

In [ ]:
# roc_curves
ludwig_job.roc_curves()

## print_models

In [ ]:
# print 
print(ludwig_job)

In [ ]:
ludwig_job.print_models()

## Predict

In [ ]:
# load_model
print('predict_model:',predict_model)
ludwig_job.load_model(predict_model)

# data
data_sample = pak.sample(data_df, predicts_size)

# predict
data_sample = ludwig_job.predict(data_sample)
data_sample

In [ ]:
if not environment['in_colab']:
    raise Stop

## Colab: Copy back

In [ ]:
# mount
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive')  

In [ ]:
if environment['in_colab']:
    !ls

In [ ]:
if environment['in_colab']:
    !ls results

In [ ]:
# results: copy to GDrive
if environment['in_colab']:
  results_path = gdrive_base + '/results'
  !cp -a results $results_path 
  !cp train_log*.csv $gdrive_base 

In [ ]:
if environment['in_colab']:
    print('Copy back manually:')
    print('1) This ipynb-File, if changed')
    print('2) results directory')
    print('3) train_log file')    
else:
    print('finished')

In [ ]:
raise Stop